In [ ]:
#Amaan Khan
#Week6 Assignment
#2024-06-08

!pip install pandas==1.3.4
!pip install modin[ray]

import pandas as pd
import numpy as np
import yaml
import os
import time
import dask.dataframe as dd
import modin.pandas as mp
import ray


num_rows = 7000000000


df = pd.DataFrame(np.random.randint(0, 100, size=(num_rows, 4)), columns=list('ABCD'))


df.to_csv('large_file.csv', index=False)


start_time = time.time()
df = pd.read_csv('large_file.csv')
print(f'Time taken to read the file using pandas: {time.time() - start_time} seconds')


start_time = time.time()
df = dd.read_csv('large_file.csv')
print(f'Time taken to read the file using Dask: {time.time() - start_time} seconds')


ray.init()
start_time = time.time()
df = mp.read_csv('large_file.csv')
print(f'Time taken to read the file using Modin and Ray: {time.time() - start_time} seconds')


df.columns = df.columns.str.replace('[#,@,&]', '')


data = {'separator': ',', 'columns': list(df.columns)}


with open('data.yaml', 'w') as file:
    yaml.dump(data, file)

with open('data.yaml', 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)

assert (data['columns'] == list(df.columns)), "Columns do not match!"

df.to_csv('data.gz', sep='|', compression='gzip', index=False)

print(f'Total number of rows: {len(df)}')
print(f'Total number of columns: {len(df.columns)}')
print(f'File size: {os.path.getsize("data.gz")} bytes')



